In [1]:
def load_namespace():
    import sys
    sys.path.insert(1,f'/wsu/home/gy/gy40/gy4065/hm.jetscapeml_source')
    sys.path.insert(1,'/content/drive/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source')
    sys.path.insert(1,'/content/drive/MyDrive/Projects/110_JetscapeMl/hm_jetscapeml_source')
    sys.path.insert(1,f'/mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source')
    sys.path.insert(1,'G:\\My Drive\\Projects\\110_JetscapeMl\\hm_jetscapeml_source')
load_namespace()


In [8]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.17.0
Keras Version: 3.5.0

Python 3.12.5 | packaged by conda-forge | (main, Aug  8 2024, 18:36:51) [GCC 12.4.0]
Pandas 2.2.2
Scikit-Learn 1.5.1
GPU is available


I0000 00:00:1724104215.595992    1858 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724104215.962351    1858 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724104215.963420    1858 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [7]:
import jet_ml.classifiers.alpha_s.preprocess_dataset as pred
(x,y_raw,y_df)=pred.preprocess_dataset_for_alpha_s(1000)
# pred.get_preprocess_dataset_info(x,y)

Dataset Preprocessor
All Path are set


2024-08-19 17:49:11.986147: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-19 17:49:12.046739: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-19 17:49:12.060981: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-19 17:49:12.226153: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-19 17:49:14.560727: W tensorflow/compiler/tf2

Aggregatring all parameters values
label_items:
 {'eloss_items': ['MMAT', 'MLBT'], 'alpha_s_items': [0.2, 0.3, 0.4], 'q0_items': [1.5, 2.0, 2.5]}
Building required params for the loading the dataset file
labels_str:
 {'eloss_items_str': 'MMAT_MLBT', 'alpha_s_items_str': '0.2_0.3_0.4', 'q0_items_str': '1.5_2.0_2.5'}
Loading the whole dataset
Extract the working column#1 for classification
dataset.x: <class 'numpy.ndarray'> 1024000 (1000, 32, 32)
dataset.y: <class 'numpy.ndarray'> 1000 (1000,)


In [9]:
y_classes=y_df.columns
y=y_df.values
display(y_classes)
display(y)

Index(['0.2', '0.3', '0.4'], dtype='object')

array([[0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       ...,
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1]])

In [10]:
import pandas as pd
df_y=pd.DataFrame(data=y,columns= [1,2,3])
display(df_y)

,1,2,3
0,0,0,1
1,0,0,1
2,1,0,0
3,0,0,1
4,0,0,1
...,...,...,...
995,0,0,1
996,0,0,1
997,0,1,0
998,0,0,1


In [11]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
k_fold=StratifiedKFold(5,shuffle=False)

out_of_sample_y=[]
out_of_sample_pred=[]
fold=0

#Must specify y StratifiedKFold for classification
for train,test in k_fold.split(x,y_raw):
    fold+=1
    print(f"Fold #{fold}")

    x_train=x[train]
    y_train=y[train]
    x_test=x[test]
    y_test=y[test]

    import jet_ml.models.test_net as test_net
    input_shape=x[0].shape
    output_shape=y.shape[1]
    activation='softmax'
    model=test_net.build_model(input_shape,output_shape,activation)
    model=test_net.compile_model(model)
    model.summary()


    batch_size=128
    epochs=1000
    monitor='val_loss' #'val_accuracy' or 'val_loss'
    model, history=test_net.train_model(model,
                                        x_train,y_train, x_test,y_test, 
                                        epochs, batch_size, monitor)
    
    pred = model.predict(x_test)

    out_of_sample_y.append(y_test)
    # raw probabilities to chosen class (highest probability)
    pred = np.argmax(pred,axis=1) 
    out_of_sample_pred.append(pred)

    # Measure this fold's accuracy
    y_compare = np.argmax(y_test,axis=1) # For accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)   
    print(f"Fold score (accuracy): {score}")

# Build the oos prediction list and calculate the error.
out_of_sample_y=np.concatenate(out_of_sample_y)
out_of_sample_pred=np.concatenate(out_of_sample_pred)
out_of_sample_y_compare=np.argmax(out_of_sample_y,axis=1)# For accuracy calculation

score=metrics.accuracy_score(out_of_sample_y_compare,out_of_sample_pred)
out_of_sample_y=pd.DataFrame(out_of_sample_y)
out_of_sample_pred=pd.DataFrame(out_of_sample_pred)

Fold #1


/home/arsi/miniconda3/envs/tensorflow/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1724104317.816423    1858 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724104317.816834    1858 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724104317.817057    1858 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724104318.107231    1

Model: "testnet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,624,963 (6.20 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,624,963 (6.20 MB)

 Trainable params: 1,624,963 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000

Epoch 1: val_loss improved from inf to 1.09841, saving model to /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/models/testnet/model.keras
7/7 - 5s - 737ms/step - accuracy: 0.3438 - loss: 1.0988 - val_accuracy: 0.3800 - val_loss: 1.0984
Epoch 2/1000

Epoch 2: val_loss improved from 1.09841 to 1.09701, saving model to /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/models/testnet/model.keras
7/7 - 1s - 79ms/step - accuracy: 0.3663 - loss: 1.0934 - val_accuracy: 0.3350 - val_loss: 1.0970
Epoch 3/1000

Epoch 3: val_loss improved from 1.09701 to 1.09584, saving model to /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/models/testnet/model.keras
7/7 - 0s - 70ms/step - accuracy: 0.4313 - loss: 1.0827 - val_accuracy: 0.3550 - val_loss: 1.0958
Epoch 4/1000

Epoch 4: val_loss did not improve from 1.09584
7/7 - 0s - 25ms/step - accuracy: 0.4325 - loss: 1.0653 - val_accuracy: 0.3950 - val_loss: 1.1007
Epoch 5/1000

Epoch 5: val_loss did not impr

In [12]:
import pandas as pd
display(score)
out_of_sample_y=pd.DataFrame(out_of_sample_y)
out_of_sample_pred=pd.DataFrame(out_of_sample_pred)

0.374

In [14]:
out_of_sample_DF=pd.concat([y_df,out_of_sample_y,out_of_sample_pred],axis=1)
import jet_ml.config as config
import os
out_of_sample_DF.to_csv(os.path.join(config.REPORTS_DIR,"alpha_s_test_net_5_fold_1000_epoch_1k.csv"),index=False)

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

In [16]:
import jet_ml.models.test_net as test_net
input_shape=x[0].shape
activation='softmax'
model=test_net.build_model(input_shape,3,activation)
model=test_net.compile_model(model)
model.summary()

/home/arsi/miniconda3/envs/tensorflow/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "testnet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,624,963 (6.20 MB)

 Trainable params: 1,624,963 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
batch_size=128
epochs=1000
monitor='val_loss' #'val_accuracy' or 'val_loss'
model, history=test_net.train_model(model,
                                    x_train,y_train, x_test,y_test, 
                                    epochs, batch_size, monitor)


Epoch 1/1000

Epoch 1: val_loss improved from inf to 1.09768, saving model to /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/models/testnet/model.keras
6/6 - 9s - 1s/step - accuracy: 0.3347 - loss: 1.0997 - val_accuracy: 0.4200 - val_loss: 1.0977
Epoch 2/1000

Epoch 2: val_loss improved from 1.09768 to 1.09540, saving model to /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/models/testnet/model.keras
6/6 - 1s - 86ms/step - accuracy: 0.3947 - loss: 1.0934 - val_accuracy: 0.3800 - val_loss: 1.0954
Epoch 3/1000

Epoch 3: val_loss improved from 1.09540 to 1.09055, saving model to /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/models/testnet/model.keras
6/6 - 1s - 88ms/step - accuracy: 0.3947 - loss: 1.0861 - val_accuracy: 0.3600 - val_loss: 1.0905
Epoch 4/1000

Epoch 4: val_loss improved from 1.09055 to 1.08627, saving model to /mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source/models/testnet/model.keras
6/6 - 1s - 100ms/step - accuracy: 0

In [18]:
import jet_ml.evaluation as eval
eval.get_accuracy_cpu(model=model,x_test=x_test,y_test=y_test)
eval.get_accuracy_gpu(model=model,x_test=x_test,y_test=y_test,batch_size=100)

Test loss 1.0849977731704712
Test accuracy: 0.335999995470047
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 241ms/step
Test accuracy: 0.36363636363636365
